In [13]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

In [14]:
df = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-r-sipenjatov-23/FINAM/tack_01/file_02.txt', sep=",")

In [15]:
df.head()

,Клиент,Дата рождения,Пол,Открытие_счета,Пополнил_счет,Дата получения лида,Источник лида
0,Клиент_30,1982-10-01,мужской,False,False,2019-03-13,Другой интернет источник
1,Клиент_71,1986-10-04,мужской,True,False,2018-02-13,Пришел в офис
2,Клиент_76,1981-07-20,мужской,True,False,2016-10-30,google
3,Клиент_137,1984-01-13,женский,True,True,2019-05-13,Пришел в офис
4,Клиент_260,1983-11-18,мужской,True,False,2016-04-21,Другой интернет источник


In [16]:
cols = {
    'Клиент':'client_id',
    'Дата рождения':'birthdate',
    'Пол' : 'sex',
    'Открытие_счета' : 'create_account',
    'Пополнил_счет' : 'top_up_account',
    'Дата получения лида':'receive_lead',
    'Источник лида':'lead_source',
}

df = df.rename(columns = cols)

In [26]:
df.head(3)

,client_id,birthdate,sex,create_account,top_up_account,receive_lead,lead_source,age
0,Клиент_30,1982-10-01,мужской,False,False,2019-03-13,Другой интернет источник,39
1,Клиент_71,1986-10-04,мужской,True,False,2018-02-13,Пришел в офис,35
2,Клиент_76,1981-07-20,мужской,True,False,2016-10-30,google,40


# Конверсионные воронки

In [27]:
def df_for_funnel(df):    
    # Получил лид
    df_received = df \
        .groupby('lead_source') \
        .agg({'client_id':'count'}) \
        .rename(columns={'client_id':'received_lead'}) \
        .sort_values('lead_source')

    # Открыл счет
    df_create_acc = df \
        .query('create_account == True') \
        .groupby('lead_source') \
        .agg({'client_id':'count'}) \
        .rename(columns={'client_id':'create_account'}) \
        .sort_values('lead_source')

    # Пополнил счет
    df_top_up = df \
        .query('create_account == True and top_up_account == True') \
        .groupby('lead_source') \
        .agg({'client_id':'count'}) \
        .rename(columns={'client_id':'top_up_account'}) \
        .sort_values('lead_source')

    df_funnel = df_received
    df_funnel['create_account'] = df_create_acc['create_account']
    df_funnel['top_up'] = df_top_up['top_up_account']

    df_funnel = df_funnel.T
    return df_funnel

In [28]:
df_funnel = df_for_funnel(df)

In [29]:
def create_funnel(df_funnel):
    
    fig = go.Figure()

    fig.add_trace(go.Funnel(
        name = 'Пришел в офис',
        y = df_funnel.index,
        x = df_funnel['Пришел в офис'],
        textinfo = "value+percent initial"))

    fig.add_trace(go.Funnel(
        name = 'Другой интернет источник',
        y = df_funnel.index,
        x = df_funnel['Другой интернет источник'],
        textinfo = "value+percent initial"))

    fig.add_trace(go.Funnel(
        name = 'yandex',
        y = df_funnel.index,
        x = df_funnel['yandex'],
        textinfo = "value+percent initial"))

    fig.add_trace(go.Funnel(
        name = 'finam.ru',
        y = df_funnel.index,
        x = df_funnel['finam.ru'],
        textinfo = "value+percent initial"))

    fig.add_trace(go.Funnel(
        name = 'google',
        y = df_funnel.index,
        x = df_funnel['google'],
        textinfo = "value+percent initial"))


    fig.add_trace(go.Funnel(
        name = 'рассылка',
        y = df_funnel.index,
        x = df_funnel['рассылка'],
        textinfo = "value+percent initial"))

    fig.add_trace(go.Funnel(
        name = 'соц.сети',
        y = df_funnel.index,
        x = df_funnel['соц.сети'],
        textinfo = "value+percent initial"))

    fig.show()

In [31]:
# Конверсионная воронка для всех пользователей
# Исходя из воронки делаем вывод, что самая большая конверсия идет по каналу 'google', но стоит учеть,
# что выборка сильно отличается по размеру с 'пришел в офис'
create_funnel(df_funnel)

# Воронки по возрасту

In [32]:
df['age'] = \
(pd.Timestamp('now') - pd.to_datetime(df['birthdate'])).astype('timedelta64[Y]').astype('int')

In [33]:
# возрастная группа 18-30 лет
# наблюдаем, что у молодых клиентов этап 'открытия счета' не отличается с остальным группами,
# а вот 'пополнение' заметно меньше(но не стоит забывать о размере выборки)
df_youth = df.query('age >= 18 & age < 30')
df_youth = df_for_funnel(df_youth)
create_funnel(df_youth)

In [34]:
# Возратсная группа 30-50 лет
df_adult = df.query('age >= 30 & age < 50')
df_adult = df_for_funnel(df_adult)
create_funnel(df_adult)

In [36]:
# Возрастная группа 50+
df_adult_plus = df.query('age >= 50')
df_adult_plus = df_for_funnel(df_adult_plus)
create_funnel(df_adult_plus)

In [37]:
# Кажется немного странным факт, что количетсво молодых и самые взрослых(50+) пришедших
# из социальных сетей равно